In [21]:
import pandas as pd
import numpy as np

#import matplotlib.pyplot as plt
#import seaborn as sns

from datetime import datetime,date, time
import timeit

import scipy.stats as st
#import statsmodels.api as sm
from pyod.models.knn import KNN

from joblib import dump, load
import os
from os import path
#import shutil
import logging

import re

In [22]:
#tiempo de ejecucion
start = timeit.default_timer()
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter(fmt='%(asctime)s - %(levelname)s - %(message)s',datefmt='%d-%m-%Y')
handler_log = logging.FileHandler("logging.log")
handler_log.setLevel(logging.DEBUG)
handler_log.setFormatter(formatter)
logger.addHandler(handler_log)

logger.info("----Inicio de Estimacion Min Max----")

In [23]:
#Path

path_libs = "libs/"
name_config = "Input/config.xlsx"

# Nuevos archivos
name_maestro_establecimientos = './input_bauti/maestro_establecimieto/maestro_establecimiento_areas.xlsx'
name_transit_time  = "./input_bauti/historico/df_historico.xlsx"
name_establecimientos = './input_bauti/coordenadas_campos/establecimientos_geolocalizados/establecimiento_geolocalizados_google.xlsx'

# Archivos antiguos.
#name_maestro_establecimientos = "./Input_fijo/maestro_establecimientos.xlsx"
#name_establecimientos = "Input/establecimientos.xlsx"



## Parametros del modelo a definir por configuracion

In [24]:
try:
    logger.info("Carga de Parametros")
    df_config = pd.read_excel(name_config)
    vel_min = df_config.loc[0,"vel_min"].astype(int)
    vel_max = df_config.loc[0,"vel_max"].astype(int)
    vel = df_config.loc[0,"vel_media"].astype(int)
    alpha = (df_config.loc[0,"confianza"]/100).astype(float)
    hs_manejo = df_config.loc[0,"hs_manejo"].astype(int)
    hs_descanso = df_config.loc[0,"hs_descanso"].astype(int)
    segmentacion =  bool(1 if str(df_config.loc[0,"segmentacion"]).strip().lower() == "si"
                            else 0)
    logger.info(f"KNN Segmentacion es: {segmentacion}")
except Exception as ex:
    logger.warning("Error carga de parametros")
    quit()

# Carga de datos

### Establecimientos y distancia

In [25]:
logger.info("Carga de Datos")
try:
    # Cargo TT a los distintos establecimientos.
    df_consolidado = pd.read_excel(name_transit_time, parse_dates=False)
    
    # Carga Establecimientos.
    df_distancias = pd.read_excel(name_establecimientos)
    
    # Maestro establecimientos.
    df_maestro_establecimientos = pd.read_excel(name_maestro_establecimientos, dtype={"latitud":"float","longitud":"float"})
    
except Exception as ex:
    logger.warning("Error al cargar archivos",exc_info=True)
    quit()

In [26]:
# Formatteo de datos.
try:
    # Transit Time.
    df_consolidado['tiempo_real'] = pd.to_timedelta(df_consolidado['tiempo_real'], errors='coerce')

    # Mascara para eliminar registros con zona, establecimiento y georreferencias iguales.
    mask_georreferencias_duplicadas = df_distancias[["zone", "establecimiento","latitud","longitud"]].duplicated()

    # Dropeo duplicados y reseteo indices.
    df_distancias.drop(df_distancias[mask_georreferencias_duplicadas].index,inplace=True)
    df_distancias = df_distancias.reset_index(drop=True)

    # Me quedo con los duplicados
    df_duplicado = df_distancias.loc[mask_georreferencias_duplicadas]

    # Me quedo con el nombre original de los establecimientos.
    establecimiento_upper = df_distancias.establecimiento.to_list()
    
    # Llevo todos los nombres de los establecimientos a minuscula.
    df_consolidado.establecimiento = df_consolidado.establecimiento.str.strip().str.lower()
    df_distancias.establecimiento = df_distancias.establecimiento.str.strip().str.lower()
    df_maestro_establecimientos.establecimiento = df_maestro_establecimientos.establecimiento.str.strip().str.lower()
    
    # Dropeo duplicados del maestro de establecimientos.
    df_maestro_establecimientos.establecimiento.drop_duplicates(inplace=True)
except:
    logger.warning("Error al cargar archivos", exc_info=True)
    quit()


In [27]:
# Establecimientos que no se encuentran cargados en el maestro de establecimientos.
mask_establecimientos_no_incluidos = df_distancias.establecimiento.isin(df_maestro_establecimientos.establecimiento)==False
mask_zonas_no_incluidas = df_distancias.zone.isin(df_maestro_establecimientos.zona)==False
mask_good_supplier_no_incluidas = df_distancias.good_supplier.isin(df_maestro_establecimientos.good_supplier)==False
mask_area_no_incluidas = df_distancias.area.isin(df_maestro_establecimientos.area)==False

mask_no_inlcuidos = mask_establecimientos_no_incluidos & mask_zonas_no_incluidas & mask_good_supplier_no_incluidas & mask_area_no_incluidas
aux = df_distancias.loc[mask_no_inlcuidos, ["zone", "establecimiento", "good_supplier", "area", "latitud", "longitud"]].copy()

# Si se encontraron establecimeintos no incluidos.
if aux.shape[0]>0:
    # Los concateno al maestro de establecimientos.
    df_maestro_establecimientos = pd.concat ([df_maestro_establecimientos, aux])
    df_maestro_establecimientos =  df_maestro_establecimientos.reset_index(drop=True)
df_maestro_establecimientos["Duplicado"] = 0

aux = df_distancias [["zone", "establecimiento", "good_supplier", "area", "latitud", "longitud"]].copy()
# Renombro para que coincidan los campos
aux.rename(columns={'zone' : 'zona'}, inplace=True)

aux = df_maestro_establecimientos.merge(aux,"left",on=["zona","establecimiento", "good_supplier", "area"])

aux.dropna(subset=["latitud_y","longitud_y"], inplace=True)

aux ["Duplicado"] = aux.apply(lambda row: 1 if ((round(row.latitud_x,9) != round(row.latitud_y,9))|
                                        (round(row.longitud_x,9) != round(row.longitud_y,9))) else 0,axis=1)

aux = aux [["zona", "establecimiento", "good_supplier", "area","latitud_y","longitud_y","Duplicado"]]
aux.rename(columns={"latitud_y":"latitud","longitud_y":"longitud"},inplace=True)

# Me quedo con los registros que tienen un error en la latitud o longitud.
aux = aux.loc[aux["Duplicado"] == 1]
# Si hay registros con errores.
if aux.shape[0]>0:
    # Los concateno al maestro de materiales.
    df_maestro_establecimientos = pd.concat ([df_maestro_establecimientos, aux]).reset_index(drop=True)

df_maestro_establecimientos.loc[df_maestro_establecimientos.Duplicado==1,"Duplicado"] = "Diferencia geolocalizacion"


In [28]:
def convert_timestamp (row):
    formato = '%d/%m/%Y %H:%M'
    
    anio = row['año']
    dia_mes_inicio = row['start_dt'].split(' ')
    dia_mes_final = row['end_dt'].split(' ')
    
    start_date = dia_mes_inicio[0] + '/' + str(anio) + ' ' + dia_mes_inicio[-1]
    end_date = dia_mes_final[0] + '/' + str(anio) + ' ' + dia_mes_final[-1]
    
    format_start_date = datetime.strptime(start_date, formato)
    format_end_date = datetime.strptime(end_date, formato)
    
    tiempo_real = format_end_date - format_start_date
    tiempo_real_secs = tiempo_real.seconds
    
    hours, remainder = divmod(tiempo_real_secs, 3600)  # 3600 seconds in an hour
    minutes, seconds = divmod(remainder, 60)   # 60 seconds in a minute
    
    return time(hours, minutes, seconds)

In [29]:
#df_consolidado['tiempo_real'] = df_consolidado.apply(lambda row: convert_timestamp(row), axis=1)
#df_consolidado['tiempo_real'] = pd.to_timedelta(df_consolidado['tiempo_real']).dt.time

### Carga de modelo de Segmentacion

In [30]:
# Antes de segmentar me fijo si el cliente envio ya dividido por subzonas.
# Si la zona se encuentra dividida por subzonas procedo a eliminarlas antes de segmentar.
if segmentacion:
    df_consolidado.subZone = df_consolidado.subZone.str.replace(r'\d*','', regex=True)
    df_distancias.zone = df_distancias.zone.str.replace(r'\d*','', regex=True)

In [31]:
# Defino funcion para obtener el tiempo de descanso en cada viaje.
# se calcula si la distancia dividida la velocidad media de 60 km supera las horas de manejo
# si supera las horas de manejo se divide la distancia por la velocidad de manejo permitida (3 hs)
# luego se multiplica por 60 para obtener el resultado en minutos y se divide por el tiempo de descanso en hs
# obteniendo los minutos de descanso para le viaje.
def duracion_descanso(row):
    # Obtengo las hs de viaje.
    hs_viaje = (row['distancia'] / vel)
    
    # Obtengo la cantidad de descansos necesarios para el viaje.
    cant_descansos = hs_viaje / hs_manejo
    # Obtengo los minutos de descanso en el viaje.
    min_descanso = cant_descansos * hs_descanso * 60 
    
    # Obtengo los minutos de viaje.
    min_viaje = hs_viaje * 60
    # Obtengo los min de manejo que se permiten.
    min_manejo = hs_manejo * 60
    
    # Si el viaje dura mas que las horas estipuladas de manejo.
    if min_viaje > min_manejo:
        return min_descanso
    else:
        return 0

In [32]:
try:
    df_distancias ["duracion_cdescanso"] = df_distancias.apply(lambda row: duracion_descanso(row),axis=1)
    
    # Obtengo la duracion del viaje con desanso incluido.
    df_distancias ["duracion_min_cdescanso"] = df_distancias["duracion_cdescanso"] + df_distancias["duracion_min"]
    
    # Selecciono los campos necesarios para la clusterizacion
    df_cluster = df_distancias[['distancia_value_mts','duracion_min_cdescanso']].copy()
    
    # Carga del modelo de clusterizacion
    kmeans = load(path.join(path_libs,"cluster_establecimientos.joblibs"))
    # Predigo y obtengo grupos.
    group = kmeans.predict(df_cluster)
    df_distancias ["group"] = group
    
    #Segementacion de los establecimientos mediante el modelo de clustering
    df_distancias["zone_group"] = ""
    
    zones = df_distancias.zone.value_counts().index.to_list()
    for zone in zones:
        groups = df_distancias.loc[df_distancias.zone == zone,"group"].value_counts().index.to_list()
        groups.sort()
        df_distancias.loc[df_distancias.zone==zone,"zone_group"] = df_distancias.loc[df_distancias.zone==zone].apply(lambda x: zone + str(groups.index(x.group)+1) if len(groups) > 1 else x.zone, axis=1) 

    resumen_segmentacion = df_distancias.pivot_table(values=["distancia","duracion_min_cdescanso"], index="group", aggfunc=["min","max"])
    resumen_segmentacion.columns = resumen_segmentacion.columns.to_flat_index()
    resumen_segmentacion.columns = ["Distancia Minima","Duracion Minima","Distancia Maxima","Duracion Maxima"]
    resumen_segmentacion = resumen_segmentacion.sort_values(by="Distancia Minima").reset_index(drop=True)
    resumen_segmentacion.reset_index(inplace=True)
    resumen_segmentacion.rename(columns={"index":"Grupo"},inplace = True)
    resumen_segmentacion["Grupo"] +=1
    resumen_segmentacion = resumen_segmentacion [["Grupo","Distancia Minima","Distancia Maxima","Duracion Minima","Duracion Maxima"]]   
except Exception as ex:
    logger.warning("Error al cargar modelo de segmentacion",exc_info=True)
    quit()

In [33]:
# Voy a agregar la informacion de los establecimientos a la informacion de los viajes.

# Me quedo unicamente con los establecimientos que estan en df_distancias.
establecimientos_considerar = df_distancias['establecimiento'].unique().tolist()
#df_consolidado = df_consolidado.loc[df_consolidado['establecimiento'].isin(establecimientos_considerar),:]

# Esta bien si despues de este merge hay mas establecimiento
df_consolidado = df_consolidado.merge(df_distancias, how="left", on=['establecimiento'])

#Mantiene la zona de segmentacion de cluster si es True caso contrario mantiene las zonas del cliente
if (segmentacion==True):
    df_consolidado.subZone = df_consolidado.zone_group
    df_distancias.zone =  df_distancias.zone_group
else:
    df_consolidado.subZone = df_consolidado.zone #valores segun el input - "coordenadas_campos"

# Preprocesamiento

### Preprocesado de Variables

In [34]:
df_consolidado.tiempo_real.dropna(axis=0,inplace=True)
df_consolidado.drop(df_consolidado.loc[df_consolidado.tiempo_real.apply(lambda x: type(x) is float)].index, axis=0,inplace=True)

#tiempo real en horas
df_consolidado["tiempo_real"] = df_consolidado.tiempo_real.apply(lambda x: ( ((x.day*24)*60) + (x.hour*60) +(x.minute)  ) 
                                                                 if isinstance(x,datetime)  else ((x.hour*60) + (x.minute)))

df_consolidado["tiempo_real"] = df_consolidado["tiempo_real"].astype(float)
df_consolidado = df_consolidado[["subZone","establecimiento", "good_supplier", "area", "distancia","tiempo_real"]]

AttributeError: 'Timedelta' object has no attribute 'hour'

In [ ]:
#Eliminar outliers
logger.info("Eliminacion de outliers")
#velocidades para eliminar  outliers
min, max = 30,110

In [ ]:
#Calculo de la Duracion media, minima y maxima

df_consolidado["duracion_cdescanso"] = df_consolidado.apply(lambda x: ((x.distancia/vel)*60),
                                                            axis=1)
df_consolidado["duracion_min_cdescanso"] = df_consolidado.apply(lambda x: ((x.distancia/max)*60),
                                                            axis=1)
df_consolidado["duracion_max_cdescanso"] = df_consolidado.apply(lambda x: ((x.distancia/min)*60),
                                                            axis=1)

In [ ]:
#Suma de horas de descanso si la duracion del viaje es mayor que la establecida para horas de manejo
df_consolidado["duracion_cdescanso"] += df_consolidado.apply(lambda x:  ((x.duracion_cdescanso)/hs_manejo)*hs_descanso
                    if (x.duracion_cdescanso > (hs_manejo*60)) else 0, axis=1)

df_consolidado["duracion_min_cdescanso"] += df_consolidado.apply(lambda x:  ((x.duracion_min_cdescanso)/hs_manejo)*hs_descanso
                    if (x.duracion_cdescanso > (hs_manejo*60)) else 0 ,  axis=1)

df_consolidado["duracion_max_cdescanso"] += df_consolidado.apply(lambda x:  ((x.duracion_max_cdescanso)/hs_manejo)*hs_descanso
                    if (x.duracion_cdescanso > (hs_manejo*60)) else 0, axis=1)

In [ ]:
#Agrego una holgura de media hora para los valores que tienen una ditancia menor a la velocidad min 
#esto es porque al dividir por un numero mas grande se hace pequeña la duracion min
#y la duracion maxima tambien es se hace pequeña
df_consolidado["duracion_max_cdescanso"] += df_consolidado.apply(lambda x: 30 if x.distancia <= vel_min else 0 ,  axis=1)

In [ ]:
#Separo los outliers encontrados
df_outliers = df_consolidado.loc[(df_consolidado.tiempo_real > df_consolidado.duracion_max_cdescanso ) ]
df_outliers = pd.concat( [df_outliers,df_consolidado.loc[(df_consolidado.tiempo_real) < (df_consolidado.duracion_min_cdescanso)]])    

df_consolidado.drop(index=df_outliers.index, inplace=True)
df_consolidado.dropna(inplace=True)

df_modelo = df_consolidado [["distancia","tiempo_real"]].copy()

In [ ]:
#Eliminando outliers mediante el analisis de todos los datos KNN
local_outlier = KNN(contamination=0.03)
local_outlier = local_outlier.fit(df_modelo)
y_pred_outlier = local_outlier.predict(df_modelo)
df_consolidado["out"] = y_pred_outlier
drop = df_consolidado.loc[df_consolidado.out == 1].index

df_consolidado.drop(drop,inplace = True)
df_consolidado.drop("out",axis=1,inplace=True)

# Intervalos de Confianza

### Tools, funciones de utilidad

In [ ]:
def ic_to_horas (minutos):
    """ 
    Descripcion
    --------------------------------
    Formatea los minutos 
    calculados para una zona en str
    de HH:mm ry rendondea los
    resultados
    
    Parametros
    --------------------------------
    minutos (float) = cantidad de
    minutos establecidos para una 
    ventana de transit time.
    """
    horas = int(divmod(minutos,60)[0])
    minutos = int(divmod(minutos,60)[1])
    horas_s = f"{horas}"
    minutos_s = f"{minutos}"
    if (minutos <= 14):
        minutos_s = f"00"
    if ( 15 <= minutos < 45):
        minutos_s = f"30"
    if ( 45 <= minutos < 60):
        minutos_s = f"00"
        horas += 1
        horas_s = f"{horas}"      
    if(horas < 10):
        horas_s = f"0{horas}"
    tiempo = f"{horas_s}:{minutos_s}"
    return tiempo

In [ ]:
def min_to_horas (minutos):
    """
    Descripcion
    --------------------------------
    Formatea los minutos 
    calculados en un str tipo
    HH:mm 
    
    Parametros
    --------------------------------
    minutos (float) = cantidad de
    minutos
    """
    horas = int(divmod(minutos,60)[0])
    minutos = int(divmod(minutos,60)[1])
    horas_s = f"{horas}"
    minutos_s = f"{minutos}"
    if(horas < 10):
        horas_s = f"0{horas}"
    if (minutos < 10):
        minutos_s = f"0{minutos}"
    tiempo = f"{horas_s}:{minutos_s}"
    return tiempo

In [ ]:
def hs_descansos (minutos):
    """
    Descripcion
    --------------------------------
    Formatea los minutos 
    calculados en un str tipo
    HH:mm 
    
    Parametros
    --------------------------------
    minutos (float) = cantidad de
    minutos
    """
    horas = int(divmod(minutos,60)[0])
    minutos = int(divmod(minutos,60)[1])
    horas_s = f"{horas}"
    minutos_s = f"{minutos}"
    if (minutos == 0):
        minutos_s = "00"
    if ( 0 < minutos <= 30):
        minutos_s = "30"
    if ( 30 < minutos <= 59):
        minutos_s = "00"
        horas += 1
    if(horas < 10):
        horas_s = f"0{horas}"
        
    tiempo = f"{horas_s}:{minutos_s}"
    return tiempo

In [ ]:
# Descansos, los descansos establecidos para los choferes corresponde actualmente a
# una hora de descanso cada 3 hs de viaje

# Esta funcion esta comentada ya que este paso ya se realizo al momento de preparar 
# los datos para el modelo de segmentacion.
'''
df_distancias ["duracion_cdescanso"] = df_distancias.apply(lambda x: 
                                                           ((x.distancia/vel)*60)/((hs_manejo))*hs_descanso
                                                           if ((x.distancia/vel)*60) > ((hs_manejo)*60)
                                                           else 0,
                                                           axis=1)
'''
logger.info("Calculo Descansos")

In [ ]:
#Horas de descanso a Horas
# pasamos los descansos expresados en minutos a horas
df_distancias ["duracion_cdescanso"] = df_distancias.duracion_cdescanso.apply(lambda x: hs_descansos (x) if x > 0 else "00:00" )

In [ ]:
#Intervalos de Confianza Estadistico x Subzona
#las velocidades medias y maximas estadisitcas se calculan apartir de la distribucion de probabilidad de la muestra
# se agrupan los tiempos de todas los establecimientos que conforman la zona
# se genera su distribucion de probabilidad
# y se calcula el min y max segun un intervalo de confianza en el cual agrupa los datos alrededor de la media
logger.info("Calculos de intervalos de confianza")

### Calculo de Intervalos de Confianza por Subzona

In [ ]:
# Vamos a considerar por un lado los establecimientos de fundadora y por el otro los de comercial.

# Para df_distancias
mask_establecimientos_fundadora_distancias = df_distancias['area'] == 'fundadora'
df_distancias_comercial = df_distancias.loc[ ~ mask_establecimientos_fundadora_distancias, : ]
df_distancias_fundadora = df_distancias.loc[ mask_establecimientos_fundadora_distancias, : ]

# Para df_consolidado
mask_establecimientos_fundadora_consolidado = df_consolidado['area'] == 'fundadora'
df_consolidado_comercial = df_consolidado.loc[ ~ mask_establecimientos_fundadora_consolidado, : ]
df_consolidado_fundadora = df_consolidado.loc[ mask_establecimientos_fundadora_consolidado, : ]

In [ ]:
# Los agrego a un diccionario para poder acceder a ellos mediante un loop.
df_distancias_dict ={'0' : df_distancias_comercial, '1' : df_distancias_fundadora}
df_consolidado_dict = {'0' : df_consolidado_comercial, '1' : df_consolidado_fundadora}

In [ ]:
try:
    # Creo un Diccionario para almacenar el resultado de esta ejecucion.
    df_subzonas_dict = {}
    
    for i in range(0,2):
        # Cargo DFs.
        df_distancias_loop = df_distancias_dict[str(i)].copy()
        df_consolidado_loop = df_consolidado_dict[str(i)].copy()
        
        subzone =  df_consolidado_loop.subZone.value_counts().index
        
        for zone in subzone:
            datos = df_consolidado_loop.loc[df_consolidado_loop.subZone == zone,"tiempo_real"]
            min, max = st.norm.interval (alpha , loc = np.mean(datos), scale = datos.std()) 
            df_consolidado_loop.loc[df_consolidado_loop.subZone == zone,"ic_min"] = min
            df_consolidado_loop.loc[df_consolidado_loop.subZone == zone,"ic_max"] = max
            if min < 14:
                min =  df_consolidado_loop.loc[df_consolidado_loop.subZone == zone,"tiempo_real"].quantile(0.2) 
                df_consolidado_loop.loc[df_consolidado_loop.subZone == zone,"ic_min"] = min

        df_consolidado_loop[["ic_min","ic_max"]] = df_consolidado_loop[["ic_min","ic_max"]].round()
        df_consolidado_loop[["ic_min","ic_max"]] = df_consolidado_loop[["ic_min","ic_max"]].astype(int)
        
        df_consolidado_loop[["ic_min","ic_max"]]
        
        df_consolidado_loop ["ic_min_hs"] = df_consolidado_loop.ic_min.apply(lambda x:  ic_to_horas(x))
        df_consolidado_loop ["ic_max_hs"] = df_consolidado_loop.ic_max.apply(lambda x:   ic_to_horas(x))
        
        df_consolidado_loop [["subZone","ic_min_hs", "ic_max_hs"]]
        
        df_subzonas_loop = df_consolidado_loop.pivot_table(values=["ic_min","ic_max"], index=["subZone","area"],aggfunc="mean").reset_index()
        
        df_subzonas_loop ["ic_min_hs"] = df_subzonas_loop.ic_min.apply(lambda x:  ic_to_horas(x))
        df_subzonas_loop ["ic_max_hs"] = df_subzonas_loop.ic_max.apply(lambda x:  ic_to_horas(x))
        
        confianza = df_consolidado_loop.pivot_table(values="establecimiento",index=["subZone","area"] ,aggfunc="count" )
        
        confianza.columns = confianza.columns.to_flat_index()
        
        confianza = confianza.reset_index()
        
        confianza.columns = ["subZone", "area", "conteo"]
        confianza["Confianza"] = confianza.apply (lambda x: "Si" if x.conteo > 30 else "No" ,axis = 1)
        
        df_subzonas_loop = df_subzonas_loop.merge(confianza , "left", on=["subZone", "area"])
        
        aux = df_distancias_loop.pivot_table(values="distancia",index=["zone", "area"] )
        
        aux.columns = aux.columns.to_flat_index()
        aux.columns = ["distancia"]
        aux = aux.reset_index()
        aux.drop("distancia",axis=1,inplace=True)
        
        df_subzonas_loop = aux.merge(df_subzonas_loop,"left",left_on=["zone","area"],right_on=["subZone","area"])
        df_subzonas_loop ["subZone"] = df_subzonas_loop.zone
        
        df_subzonas_loop.Confianza.fillna("Nuevo", inplace=True)
        df_subzonas_loop.fillna("-",inplace=True)
        
        # Agrego el resultado de esta ejecucion del loop en el diccionario.
        df_subzonas_dict[str(i)] = df_subzonas_loop
except Exception as ex:
    logger.warning("Error al calcular min max zonas estadistico", exc_info=True)
    quit()

### Intervalos de Confianza Estadistico x Establecimiento

In [ ]:
# Calculo de Intervalos de Confianza por Establecimiento
try:
    # Creo un Diccionario para almacenar el resultado de esta ejecucion.
    df_establecimiento_dict = {}

    for i in range(0,2):
        # Cargo DFs.
        df_distancias_loop = df_distancias_dict[str(i)].copy()
        df_consolidado_loop = df_consolidado_dict[str(i)].copy()
        
        establecimientos = df_distancias_loop.establecimiento.value_counts().index
        
        for zone in establecimientos:
            datos = df_consolidado_loop.loc[df_consolidado_loop.establecimiento == zone,"tiempo_real"]
            min, max = st.norm.interval (alpha , loc = np.mean(datos), scale = datos.std()) 
            df_consolidado_loop.loc[df_consolidado_loop.establecimiento == zone,"ic_min_establecimiento"] = min
            df_consolidado_loop.loc[df_consolidado_loop.establecimiento == zone,"ic_max_establecimiento"] = max 
            
        df_consolidado_loop ["ic_esta_min_hs"] = df_consolidado_loop.ic_min_establecimiento.apply(lambda x:  ic_to_horas(x) )
        df_consolidado_loop ["ic_esta_max_hs"] = df_consolidado_loop.ic_max_establecimiento.apply(lambda x:   ic_to_horas(x) )
        
        df_establecimiento_loop = df_consolidado_loop.pivot_table(values=["ic_min_establecimiento","ic_max_establecimiento","tiempo_real"], 
                                                        index=["establecimiento", "subZone", "area", "good_supplier"],
                                                        aggfunc = {
                                                            "ic_min_establecimiento" : "mean",
                                                            "ic_max_establecimiento" : "mean",
                                                            "tiempo_real" : "count"
                                                        } ).reset_index()
        
        df_establecimiento_loop ["ic_min_hs"] = df_establecimiento_loop.ic_min_establecimiento.apply(lambda x:  min_to_horas(x))
        df_establecimiento_loop ["ic_max_hs"] = df_establecimiento_loop.ic_max_establecimiento.apply(lambda x:  min_to_horas(x))
        
        df_establecimiento_loop = df_establecimiento_loop [["subZone", "establecimiento","good_supplier", "area", "ic_min_hs","ic_max_hs","tiempo_real"]].copy()
        
        df_establecimiento_loop ["Confianza"] = df_establecimiento_loop.tiempo_real.apply(lambda x: "Si" if x > 30 else "No")
        
        df_establecimiento_loop = df_distancias_loop.merge(
            df_establecimiento_loop,
            how="left",
            left_on = ['zone', 'establecimiento', 'good_supplier', 'area'],
            right_on = ['subZone', 'establecimiento', 'good_supplier', 'area']
            )
        
        df_establecimiento_loop = df_establecimiento_loop[["subZone", "establecimiento","good_supplier", "area", "ic_min_hs","ic_max_hs","Confianza"]]
        
        df_establecimiento_loop.ic_min_hs.fillna("-",inplace=True)
        df_establecimiento_loop.ic_max_hs.fillna("-",inplace=True)
        df_establecimiento_loop.Confianza.fillna("Nuevo",inplace=True)
        
        df_distancias_loop_merge_descanso = df_distancias_loop[['establecimiento','duracion_cdescanso']]
        df_establecimiento_loop = df_establecimiento_loop.merge(df_distancias_loop_merge_descanso, how='left', on='establecimiento')
        
        df_establecimiento_loop.columns = ["Zona", "Establecimiento","Good Supplier", "Area", "T.T Min", "T.T Max","Confianza","Descanso"]
        
        # Agrego el resultado de esta ejecucion del loop en el diccionario.
        df_establecimiento_dict[str(i)] = df_establecimiento_loop
        
except Exception as ex:
    logger.warn("Error al calcular los min max por establecimiento estadistico", exc_info=True)
    quit()

In [ ]:
#Intervalo de confianza algoritmico Establecimiento.
try:
    # Creo un Diccionario para almacenar el resultado de esta ejecucion.
    df_subzonas_2_dict = {}

    for i in range(0,2):
        # Cargos DFs.
        df_distancias_loop = df_distancias_dict[str(i)].copy()
        df_establecimiento_loop = df_establecimiento_dict[str(i)].copy()
        
        df_subzonas_2_loop = df_distancias_loop[["zone", "establecimiento", "good_supplier", "area","distancia","duracion_cdescanso"]].copy()
        
        df_subzonas_2_loop["duracion_min"] = ((df_subzonas_2_loop["distancia"] / vel_max) * 60)
        df_subzonas_2_loop["duracion_min"] += df_subzonas_2_loop.apply(lambda x:  ((x["duracion_min"])/hs_manejo)*hs_descanso
                            if (x["distancia"] > (hs_manejo*60)) 
                            else x["duracion_min"], axis=1)
        
        df_subzonas_2_loop["duracion_max"] = ((df_subzonas_2_loop["distancia"]/vel_min)*60)
        df_subzonas_2_loop["duracion_max"] += df_subzonas_2_loop.apply(lambda x:  ((x["duracion_max"])/hs_manejo)*hs_descanso
                            if (x["distancia"] > (hs_manejo*60))
                            else x["duracion_max"],
                            axis=1)
        
        df_subzonas_2_loop ["ic_hs_min"] = df_subzonas_2_loop["duracion_min"].apply(lambda x:  min_to_horas(x) )
        df_subzonas_2_loop ["ic_hs_max"] = df_subzonas_2_loop["duracion_max"].apply(lambda x:   min_to_horas(x) )
        
        df_subzonas_2_loop = df_subzonas_2_loop  [["zone", "establecimiento", "good_supplier", "area","ic_hs_min", "ic_hs_max","duracion_cdescanso"]]
        df_subzonas_2_loop.columns = ["Zona", "Establecimiento", "Good Supplier", "Area","T.T Min","T.T Max","Descanso"]
        
        
        # Agrego el resultado de esta ejecucion del loop en el diccionario.
        df_subzonas_2_dict[str(i)] = df_subzonas_2_loop

except Exception as ex:
    logger.warning("Error al calcular min max algoritmico por establecimiento")
    quit()

In [ ]:
# Intevalo de Confianza algoritmico por zona
try:
    # Creo un Diccionario para almacenar el resultado de esta ejecucion.
    df_subzonas_algoritmico_dict = {}
        
    for i in range(0,2):
        # Cargos DFs.
        df_distancias_loop = df_distancias_dict[str(i)].copy()
        
        subzonas_algoritmico_loop =  df_distancias_loop.pivot_table(
            values="distancia", 
            index=["zone", 'area'],
            aggfunc=["min","max"]
        )
        
        subzonas_algoritmico_loop.columns = subzonas_algoritmico_loop.columns.to_flat_index()
        subzonas_algoritmico_loop = subzonas_algoritmico_loop.reset_index()
        subzonas_algoritmico_loop.columns = ["zone", "area", "distancia_min","distancia_max"]
        
        subzonas_algoritmico_loop["duracion_min"] = ((subzonas_algoritmico_loop["distancia_min"] / vel_max)*60)
        subzonas_algoritmico_loop["duracion_min"] += subzonas_algoritmico_loop.apply(lambda x:  ((x["duracion_min"])/hs_manejo)*hs_descanso
                                                                        if (x["distancia_min"] > (hs_manejo*60)) 
                                                                        else x["duracion_min"], 
                                                                        axis=1)
        
        subzonas_algoritmico_loop["duracion_max"] = ((subzonas_algoritmico_loop["distancia_max"]/vel_min)*60)
        subzonas_algoritmico_loop["duracion_max"] += subzonas_algoritmico_loop.apply(lambda x:  ((x["duracion_max"])/hs_manejo)*hs_descanso
                            if (x["distancia_max"] > (hs_manejo*60))
                            else x["duracion_max"],
                            axis=1)
        
        subzonas_algoritmico_loop.loc[subzonas_algoritmico_loop.duracion_min < 14 ,"duracion_min"] = 15 
        subzonas_algoritmico_loop ["ic_hs_min"] = subzonas_algoritmico_loop["duracion_min"].apply(lambda x:  ic_to_horas(x) )
        subzonas_algoritmico_loop ["ic_hs_max"] = subzonas_algoritmico_loop["duracion_max"].apply(lambda x:   ic_to_horas(x) )
        subzonas_algoritmico_loop = subzonas_algoritmico_loop[["zone", "area", "ic_hs_min","ic_hs_max"]]
        subzonas_algoritmico_loop.columns = ["Zona", "Area", "T.T Min", "T.T Max"]
        
        # Agrego el resultado de esta ejecucion del loop en el diccionario.
        df_subzonas_algoritmico_dict[str(i)] = subzonas_algoritmico_loop
except Exception as ex:
    logger.warning("Error al calcular min max algoritmico por zona")
    quit()

Ahora voy a imputar valores que pudieran haber quedado en nulo ya que no se tenia los datos suficientes como para poder obtener el TT.


In [ ]:
# Creo una funcion para imputar los valores de TT min y max segun el metodo algoritmico.
# Se hace asi ya que hay valores que no pudieron ser calculados por falta de datos.
def imputacion_confianza_nueva_zona (df_confianza, df_algoritmico):
    # Obtengo las zonas con confianza nueva
    mask_zonas_nuevas = df_confianza['Confianza'] == 'Nuevo'
    lista_zonas_nuevas = df_confianza.loc[mask_zonas_nuevas, 'zone']
    
    # Itero por cada una de esas zonas
    for zona_nueva in lista_zonas_nuevas:
        # Creo mascaras para considerar la zona del loop en ambos dfs.
        mask_zona_confianza = df_confianza['zone'] == zona_nueva
        mask_zona_algoritmico = df_algoritmico['Zona'] == zona_nueva
        
        # imputo valores para esa zona.
        df_confianza.loc[mask_zona_confianza, ['ic_min_hs', 'ic_max_hs']] = df_algoritmico.loc[mask_zona_algoritmico, ['T.T Min', 'T.T Max']].values
    return df_confianza


In [ ]:
# Hago lo mismo pero para imputar segun establecimiento.
def imputacion_confianza_nuevo_establecimiento (df_confianza, df_algoritmico):
    # Obtengo los establecimeintos con confianza nueva
    mask_establecimientos_nuevos = df_confianza['Confianza'] == 'Nuevo'
    lista_establecimientos_nuevos = df_confianza.loc[mask_establecimientos_nuevos, 'Establecimiento']
    
    cols_sobreescribir = ['Zona', 'T.T Min', 'T.T Max']
    # Itero por cada uno de estos establecimientos.
    for establecimiento in lista_establecimientos_nuevos:
        # Creo mascaras para seleccionar el establecimiento de ambos dfs.
        mask_establecimiento_confianza = df_confianza['Establecimiento'] == establecimiento
        mask_establecimiento_algoritmico = df_algoritmico['Establecimiento'] == establecimiento
        
        # Imputo valores.
        df_confianza.loc[mask_establecimiento_confianza, cols_sobreescribir] = df_algoritmico.loc[mask_establecimiento_algoritmico, cols_sobreescribir].values
    return df_confianza

In [ ]:
for i in range(0,2):
    # Imputo Zonas
    df_subzonas_dict[str(i)] = imputacion_confianza_nueva_zona(df_subzonas_dict[str(i)], df_subzonas_algoritmico_dict[str(i)])

    # Imputo Establecimientos
    df_establecimiento_dict[str(i)] = imputacion_confianza_nuevo_establecimiento(df_establecimiento_dict[str(i)] , df_subzonas_2_dict[str(i)])

### Concatenacion de resultados

In [ ]:
df_subzonas = pd.concat([df_subzonas_dict['0'], df_subzonas_dict['1']], ignore_index=True)
df_establecimiento = pd.concat([df_establecimiento_dict['0'], df_establecimiento_dict['1']], ignore_index=True)
df_subzonas_2 = pd.concat([df_subzonas_2_dict['0'], df_subzonas_2_dict['1']], ignore_index=True)
subzonas_algoritmico = pd.concat([df_subzonas_algoritmico_dict['0'], df_subzonas_algoritmico_dict['1']], ignore_index=True)


### Output

In [ ]:
logger.info("Output")
def output_writer (output_name, sheet_names, df, title):
    """
    Descripcion
    -------------------------------------------------------
    Funcion que da un formato de tabla prefinido para excel
    
    Parametros
    ------------------------------------------------------
    output_name = str, Path del archivo
    sheet_names = list, Nombres de las hojas 1 por DataFrame
    df = list, DataFrame para guardar
    title = list, Titulos de cada hoja
    """
    result = 'Exito al Guardar'
    try:
        writer = pd.ExcelWriter(output_name, engine = "xlsxwriter")

        for i in range(len(df)):
            df[i].to_excel(writer, sheet_name=sheet_names[i], startrow=2, header=False, index=False)

            workbook = writer.book
            worksheet = writer.sheets[sheet_names[i]]

            max_row, max_col = df[i].shape

            header_format = workbook.add_format({'font_name' : 'Times New Roman',
                                                 'bold': True, 
                                                 'font_color': 'black',
                                                 'font_size' : 14,
                                                 'align' : 'center',
                                                 'valign' : 'vcenter',
                                                 'border' : 1
                                                })

            cell_format = workbook.add_format({'font_name' : 'Times New Roman',
                                               'font_color' : 'black',
                                               'font_size' : 12,
                                               'align' : 'center',
                                               'valign' : 'vcenter',
                                              })

            title_format = workbook.add_format({'font_name' : 'Times New Roman',
                                          'bold': True, 
                                          'font_color': 'black',
                                          'font_size' : 14,
                                          'align' : 'center',
                                          'valign' : 'vcenter',
                                          'border' : 1,
                                          'bg_color' : "#4F81BD"
                                                })

            columns_settings = [{'header': column, 
                                 'header_format': header_format
                                } for column in df[i].columns]

            worksheet.add_table(1, 0, max_row+1, max_col - 1, {'columns': columns_settings
                                                                })
            worksheet.set_column(first_col = 0, last_col = max_col -1 , width = 15, cell_format = cell_format)

            worksheet.set_row (1,30)

            worksheet.merge_range(0,0,0,max_col-1, title[i],title_format)
            worksheet.set_row (0,30)

        #writer.save()
        writer.close()
    except Exception as ex:
        result = 'Error al Guardar'+ str(ex.__class__) 
        logger.warning("Error al guardar archivos de salida",exc_info=True)

    print (result)

In [ ]:
resumen_segmentacion["Duracion Minima"] = resumen_segmentacion["Duracion Minima"].apply(lambda x: min_to_horas(x))
resumen_segmentacion["Duracion Maxima"] = resumen_segmentacion["Duracion Maxima"].apply(lambda x: min_to_horas(x))

## Formateo del output.

In [ ]:
# Vamos a separar la letra de las zonas de el numero de la subzona clusterizada
# si es que se clusterizo. 
df_subzonas = df_subzonas [["subZone","area","ic_min_hs","ic_max_hs","Confianza"]].copy()

# Creo patron para detectar si la zona cuenta con un numero que indica la subzona.
pattern = r'[a-zA-Z]\d'

# Booleano que detecta la presencia de numeros en la zona.
condition_subzones = df_subzonas['subZone'].str.contains(pattern).any()

# Si la zona esta clusterizada.
if condition_subzones:
    # Separo letras de numeros
    df_subzonas[['zone', 'subZone']] = df_subzonas['subZone'].str.extract(r'([A-Z]+)(\d*)')
    # Reordeno DF.
    df_subzonas = df_subzonas[['zone', 'subZone', 'area', 'ic_min_hs', 'ic_max_hs', 'Confianza']]
    # Reemplazo valores vacios.
    df_subzonas['subZone'].replace('', '-', inplace=True)
    # Cambio de nombre las columnas.
    df_subzonas.columns = ["Zona","SubZona", "Area", "T.T Min", "T.T Max","Confianza"]
else:
    # Si no se detecta la presencia de numeros en la zona.
    df_subzonas.columns = ["Zona","Area", "T.T Min", "T.T Max","Confianza"]

In [ ]:
# Hago lo mismo para el modelo algoritmico.
# Booleano que detecta la presencia de numeros en la zona.
subzonas_algoritmico = subzonas_algoritmico.copy() # Para error de copia.

condition_subzones = subzonas_algoritmico['Zona'].str.contains(pattern).any()

# Si la zona esta clusterizada.
if condition_subzones:
    # Separo letras de numeros
    subzonas_algoritmico[['zone', 'subZone']] = subzonas_algoritmico['Zona'].str.extract(r'([A-Z]+)(\d*)')
    # Reordeno DF.
    subzonas_algoritmico = subzonas_algoritmico[['zone', 'subZone', 'Area', 'T.T Min', 'T.T Max']]
    # Reemplazo valores vacios.
    subzonas_algoritmico['subZone'].replace('', '-', inplace=True)
    # Cambio de nombre las columnas.
    subzonas_algoritmico.columns = ["Zona","SubZona","Area","T.T Min", "T.T Max"]

In [ ]:
# Ahora por establecimiento.
df_establecimiento = df_establecimiento.copy() # Para error de copia.
# Booleano que detecta la presencia de numeros en la zona.
condition_subzones = df_establecimiento['Zona'].str.contains(pattern).any()

# Si la zona esta clusterizada.
if condition_subzones:
    # Separo letras de numeros
    df_establecimiento[['zone', 'subZone']] = df_establecimiento['Zona'].str.extract(r'([A-Z]+)(\d*)')
    # Reordeno DF.
    df_establecimiento = df_establecimiento[[
        'zone', 
        'subZone', 
        'Establecimiento', 
        'Good Supplier', 
        'Area', 
        'T.T Min', 
        'T.T Max', 
        'Confianza', 
        'Descanso'
        ]]
    
    # Reemplazo valores vacios.
    df_establecimiento['subZone'].replace('', '-', inplace=True)
    # Cambio de nombre las columnas.
    df_establecimiento.rename(columns={'zone' : 'Zona', 'subZone' : 'SubZona'}, inplace=True)

In [ ]:
# Ahora por establecimiento del modelo constante.
df_subzonas_2 = df_subzonas_2.copy() # Para error de copia.
# Booleano que detecta la presencia de numeros en la zona.
condition_subzones = df_subzonas_2['Zona'].str.contains(pattern).any()

# Si la zona esta clusterizada.
if condition_subzones:
    # Separo letras de numeros
    df_subzonas_2[['zone', 'subZone']] = df_subzonas_2['Zona'].str.extract(r'([A-Z]+)(\d*)')
    # Reordeno DF.
    df_subzonas_2 = df_subzonas_2[[
        'zone', 
        'subZone', 
        'Establecimiento', 
        'Good Supplier', 
        'Area', 
        'T.T Min', 
        'T.T Max', 
        'Descanso'
        ]]
    
    # Reemplazo valores vacios.
    df_subzonas_2['subZone'].replace('', '-', inplace=True)
    # Cambio de nombre las columnas.
    df_subzonas_2.rename(columns={'zone' : 'Zona', 'subZone' : 'SubZona'}, inplace=True)

In [ ]:
# Ahora vamos a formatear el Df de informacionde establecimientos.

# Booleano que detecta la presencia de numeros en la zona.
condition_subzones = df_distancias['zone'].str.contains(pattern).any()

# Si la zona esta clusterizada.
if condition_subzones:
    # Separo letras de numeros
    df_distancias[['zone', 'subZone']] = df_distancias['zone'].str.extract(r'([A-Z]+)(\d*)')
    # Reemplazo valores vacios.
    df_distancias['subZone'].replace('', '-', inplace=True)
    # Selecciono columnas.
    df_distancias = df_distancias [[
        'establecimiento',
        'zone',
        'subZone',
        'good_supplier',
        'area',
        "state",
        "city",
        "latitud",
        "longitud",
        "distancia",
        "duracion",
        "duracion_cdescanso"
    ]]
    # Renombro zonas.
    df_distancias.columns = [
        'Establecimiento',
        "Zona",
        'SubZona',
        'Good Supplier',
        'Area',
        'Provincia',
        'Ciudad',
        'Latitud',
        'Longitud',
        'Distancia a Planta',
        'Duracion Google',
        'Descanso'
    ]
else:
    # Si no se detecta la presencia de numeros en la zona.
    df_distancias = df_distancias [[
        "establecimiento",
        "zone",
        'good_supplier',
        'area',
        "state",
        "city",
        "latitud",
        "longitud",
        "distancia",
        "duracion",
        "duracion_cdescanso"
    ]]

    df_distancias.columns = [
        "Establecimiento",
        "Zona",
        'Good Supplier',
        'Area',
        "Provincia",
        "Ciudad",
        "Latitud",
        "Longitud",
        "Distancia a Planta",
        "Duracion Google",
        "Descanso"
    ]

In [ ]:
# LLevo los nombres de los establecimientos a mayuscula.
df_establecimiento['Establecimiento'] = df_establecimiento['Establecimiento'].apply(lambda x: x.title())
df_subzonas_2['Establecimiento'] = df_subzonas_2['Establecimiento'].apply(lambda x: x.title())
df_distancias['Establecimiento'] = df_distancias['Establecimiento'].apply(lambda x: x.title())

## Salida

In [ ]:
fecha = str(date.today())
dir_out = "Output"
existe_directorio = False

In [ ]:
try:
    for directorio in os.listdir(dir_out):
        if directorio == fecha:
            existe_directorio = True
            dir_out = os.path.join (dir_out,fecha)
            break
    if existe_directorio == False:
        dir_out = os.path.join (dir_out,fecha)
        os.mkdir(dir_out)

    last = 0
    output_name = ""
    if (len (os.listdir(dir_out)) == 0):
        output_name = fecha+"-"+"ESTIMACIONTTMINMAX_0.xlsx"
    else:
        for file in os.listdir(dir_out):
            if file != ('.ipynb_checkpoints'):
                num = int(file.split("_")[1].split(".")[0])
                if num > last:
                    last = num
        output_name = fecha+"-"+"ESTIMACIONTTMINMAX_{:d}".format(last+1)+'.xlsx'
    output_name = os.path.join(dir_out,output_name)

    os.listdir(dir_out)

    dfs = [
        df_subzonas,
        subzonas_algoritmico,
        df_establecimiento,
        df_subzonas_2,
        df_distancias,
        resumen_segmentacion,
        df_maestro_establecimientos
    ]
    
    sheet_names = [
        "T.T Zonas Est",
        "T.T Zonas Const",
        "T.T Establecimientos Est",
        "T.T Establecimientos Const",
        "Establecimientos",
        "Segmentacion",
        "Maestro"
    ]
    
    titles = [
        "T.T Zonas Estadistico",
        "T.T Zonas Cosntante",
        "T.T Establecimientos Estadistico",
        "T.T Establecimientos Constante",
        "Datos Establecimientos",
        "Resumen Segmentacion",
        "Establecimientos Historicos"
    ]
    
    output_writer(output_name,sheet_names,dfs,titles)

    output_writer(os.path.join("Output","duplicados.xlsx"),["Duplicado"],[df_duplicado],["Establecimientos Duplicados"])

    index_drop = df_maestro_establecimientos.loc[df_maestro_establecimientos["Duplicado"]!=0].index
    df_maestro_establecimientos.drop(index_drop,inplace=True)
    df_maestro_establecimientos.to_excel(name_maestro_establecimientos,index=False)
except Exception as ex:
    logger.warning("Error al guardar los archivos",exc_info=True)
    quit()

Exito al Guardar
Exito al Guardar


c:\Users\TALIGENT\anaconda3\envs\transit_time_bayer\lib\site-packages\xlsxwriter\worksheet.py:3261: UserWarning: Must have at least one data row in in add_table()
  warn("Must have at least one data row in in add_table()")


In [ ]:
# #Movemos los archivos a procesados
# try:
#     if (hay_archivos_establecimientos_nuevo):
#         os.rename(archivo_establecimientos_nuevo, archivo_establecimientos_nuevo.replace("PENDIENTE","PROCESADO"))

# except Exception as ex:
#     logger.warning("Error al mover los archivos a procesados.",exc_info=True)
#     quit()

In [ ]:
# # Ejecutamos el envío por Mail
# try:
#     os.system("python Mail.py")

# except Exception as ex:
#     logger.warning("Error al enviar los archivos por mail.",exc_info=True)
#     quit()

In [ ]:
stop = timeit.default_timer()
logger.info(f"Fin de Ejecucion de ejecucion {stop-start}")